In [131]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model

from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [132]:
img_width = 224
img_height = 224
input_shape = (img_width, img_height,3)
train_data_dir = '../input/oai-knee/KneeXrayData/ClsKLData/kneeKL224/train'
validation_data_dir = '../input/oai-knee/KneeXrayData/ClsKLData/kneeKL224/val'
train_samples = 5778
validation_samples = 826
epochs = 50
batch_size = 20

In [133]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [134]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    )

Found 5778 images belonging to 5 classes.


In [135]:

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    )

Found 826 images belonging to 5 classes.


model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # Output convert into one dimension layer and will go to Dense layer
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))
from keras import optimizers
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=.0001),
              metrics=['accuracy'])

In [140]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model

from keras.optimizers import SGD
import keras
restnet = ResNet50(include_top="False", weights="imagenet", input_shape=(224,224,3))
flat1 = Flatten()(restnet.layers[-1].output)

# define new model
restnet = Model(inputs=restnet.inputs, outputs=flat1)
opt = SGD(lr=0.1)
restnet.summary()

Model: "functional_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_35[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________

In [141]:

model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))
model.compile(loss='CategoricalCrossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()


Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_31 (Functional)   (None, 1000)              25636712  
_________________________________________________________________
dense_80 (Dense)             (None, 512)               512512    
_________________________________________________________________
dropout_40 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_41 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 5)                 2565      
Total params: 26,414,445
Trainable params: 26,361,325
Non-trainable params: 53,120
____________________________________

In [142]:

answ = model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size 
    )

Epoch 1/50
288/288 [==============================] - 99s 343ms/step - loss: 1.4129 - accuracy: 0.3941 - val_loss: 1.4074 - val_accuracy: 0.3976
Epoch 2/50
288/288 [==============================] - 99s 345ms/step - loss: 1.4025 - accuracy: 0.3956 - val_loss: 1.4072 - val_accuracy: 0.3951
Epoch 3/50
216/288 [=====================>........] - ETA: 23s - loss: 1.3697 - accuracy: 0.4062

KeyboardInterrupt: 